In [1]:
import unittest
import pandas as pd
import numpy as np
from genome_tags_pipeline import GenomeTagDataPipeline
import tempfile
import os

In [2]:


class TestGenomeTagDataPipeline(unittest.TestCase):

    def setUp(self):
        # Créer un DataFrame de test
        self.test_data = pd.DataFrame({
            'tagId': [1, 2, 3, 4, 5],
            'tag': ['action', 'comedy', 'drama', 'sci-fi', 'romance']
        })
        
        # Créer un fichier CSV temporaire
        self.temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
        self.test_data.to_csv(self.temp_file.name, index=False)
        self.temp_file.close()  # Fermer explicitement le fichier
        
        # Initialiser le pipeline avec le fichier temporaire
        self.pipeline = GenomeTagDataPipeline(self.temp_file.name)

    def tearDown(self):
        # Supprimer le fichier temporaire après chaque test
        try:
            os.unlink(self.temp_file.name)
        except PermissionError:
            pass  # Ignorer l'erreur si le fichier ne peut pas être supprimé

    def test_load_data(self):
        self.pipeline.load_data()
        self.assertIsNotNone(self.pipeline.data)
        self.assertEqual(len(self.pipeline.data), 5)
        self.assertEqual(list(self.pipeline.data.columns), ['tagId', 'tag'])

    def test_clean_data_no_changes(self):
        self.pipeline.load_data()
        original_data = self.pipeline.data.copy()
        self.pipeline.clean_data()
        pd.testing.assert_frame_equal(self.pipeline.data, original_data)

    def test_clean_data_with_duplicates(self):
        # Ajouter une ligne dupliquée
        self.test_data = pd.concat([self.test_data, self.test_data.iloc[[0]]])
        self.test_data.to_csv(self.temp_file.name, index=False)
        
        self.pipeline.load_data()
        self.pipeline.clean_data()
        self.assertEqual(len(self.pipeline.data), 5)

    def test_clean_data_with_missing_values(self):
        # Ajouter une ligne avec des valeurs manquantes
        missing_row = pd.DataFrame([[None, None]], columns=self.test_data.columns)
        self.test_data = pd.concat([self.test_data, missing_row], ignore_index=True)
        self.test_data.to_csv(self.temp_file.name, index=False)
        
        self.pipeline.load_data()
        self.pipeline.clean_data()
        self.assertEqual(len(self.pipeline.data), 5)

    def test_calculate_statistics(self):
        self.pipeline.load_data()
        self.pipeline.calculate_statistics()
        self.assertIn('tag_length', self.pipeline.data.columns)
        self.assertEqual(self.pipeline.data['tag_length'].tolist(), [6, 6, 5, 6, 7])

    def test_describe_data(self):
        self.pipeline.load_data()
        self.pipeline.calculate_statistics()
        # Vérifier que describe_data ne lève pas d'exception
        try:
            self.pipeline.describe_data()
        except Exception as e:
            self.fail(f"describe_data a levé une exception inattendue {e}")

    def test_get_data(self):
        self.pipeline.load_data()
        data = self.pipeline.get_data()
        self.assertIsInstance(data, pd.DataFrame)
        self.assertEqual(len(data), 5)

    

In [3]:
def run_tests():
    # Créer une suite de tests
    suite = unittest.TestLoader().loadTestsFromTestCase(TestGenomeTagDataPipeline)
    
    # Créer un runner de test
    runner = unittest.TextTestRunner(verbosity=2)
    
    # Exécuter les tests
    result = runner.run(suite)
    
    # Afficher un résumé
    print(f"\nNombre total de tests : {result.testsRun}")
    print(f"Nombre de tests réussis : {result.testsRun - len(result.failures) - len(result.errors)}")
    print(f"Nombre d'échecs : {len(result.failures)}")
    print(f"Nombre d'erreurs : {len(result.errors)}")

# Exécuter les tests
run_tests()

test_calculate_statistics (__main__.TestGenomeTagDataPipeline.test_calculate_statistics) ... ok
test_clean_data_no_changes (__main__.TestGenomeTagDataPipeline.test_clean_data_no_changes) ... ok
test_clean_data_with_duplicates (__main__.TestGenomeTagDataPipeline.test_clean_data_with_duplicates) ... ok
test_clean_data_with_missing_values (__main__.TestGenomeTagDataPipeline.test_clean_data_with_missing_values) ... ok
test_describe_data (__main__.TestGenomeTagDataPipeline.test_describe_data) ... ok

Données chargées avec succès.
Statistiques calculées avec succès.
Données chargées avec succès.
Aucune valeur manquante détectée.
Aucun doublon détecté.
Données nettoyées avec succès.
Données chargées avec succès.
Aucune valeur manquante détectée.
1 doublons supprimés.
Données nettoyées avec succès.
Données chargées avec succès.
Valeurs manquantes détectées:
tagId    1
tag      1
dtype: int64
Lignes avec valeurs manquantes supprimées.
Aucun doublon détecté.
Données nettoyées avec succès.
Données chargées avec succès.
Statistiques calculées avec succès.

Description des données traitées:
Nombre total de genome tags: 5
Nombre de tags uniques: 5

Statistiques des tagId:
count    5.000000
mean     3.000000
std      1.581139
min      1.000000
25%      2.000000
50%      3.000000
75%      4.000000
max      5.000000
Name: tagId, dtype: float64

Statistiques de la longueur des tags:
count    5.000000
mean     6.000000
std      0.707107
min      5.000000
25%      6.000000
50%      6.000000
75%  


test_get_data (__main__.TestGenomeTagDataPipeline.test_get_data) ... ok
test_load_data (__main__.TestGenomeTagDataPipeline.test_load_data) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.320s

OK


Données chargées avec succès.
Données chargées avec succès.

Nombre total de tests : 7
Nombre de tests réussis : 7
Nombre d'échecs : 0
Nombre d'erreurs : 0


In [ ]:
# Créer une instance de la classe de test
test_instance = TestGenomeTagDataPipeline()

# Exécuter setUp
test_instance.setUp()

# Exécuter un test spécifique
test_instance.test_load_data()

# Exécuter tearDown
test_instance.tearDown()

print("Test exécuté avec succès.")